### 필요 라이브러리

In [1]:
import numpy as np
import pandas as pd 

pd.options.display.max_columns = None
pd.options.display.max_rows = None

import tensorflow as tf 

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

2023-04-30 02:01:39.673707: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 02:01:40.516239: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Local 환경의 파일 불러오기 

In [2]:
# Font 
FONT_PATH = "/home/jupyter/Fonts/NanumSquareR.ttf"

plt.rcParams['axes.unicode_minus']=False
plt.rcParams['font.family'] = "NanumGothic"

In [3]:
# 경로 
file_path = "/home/jupyter/직업교육/"

In [4]:
# 파일명
file = "education_tokenize_data.csv"

In [5]:
# csv 불러오기 
data = pd.read_csv(file_path + file)

In [6]:
data.shape

(10692, 5)

In [7]:
data.head()

,번호,교육명,clean_sentence,okt,mecab
0,31622628,"[주거]초록쉼표, 식물과 함께하는 일상",주거 초록쉼표 식물과 함께하는 일상,"['주거', '초록', '쉼표', '식물', '일상']","['주거', '초록', '쉼표', '식물', '일상']"
1,32138091,[맞춤형일자리]서류평가관 활동 설명회,맞춤형일자리 서류평가관 활동 설명회,"['맞춤', '일자리', '서류', '평가', '관', '활동', '설명', '회']","['맞춤', '일자리', '서류', '평가', '관', '활동', '설', '명회']"
2,31623735,[건강]몸신의 허리통증 없애는 비결,건강 몸신의 허리통증 없애는 비결,"['건강', '몸', '신의', '허리', '통증', '비결']","['건강', '몸신', '허리', '통증', '비결']"
3,31623329,[건강]국민연금공단 연계 '두근두근 뇌운동(치매의 이해와 예방)',건강 국민연금공단 연계 두근두근 뇌운동 치매의 이해와 예방,"['건강', '국민연금', '공단', '계', '뇌', '운동', '치매', '이해...","['건강', '국민', '연금', '공단', '연계', '뇌', '운동', '치매'..."
4,32130888,[일자리센터 연계]50+일자리 상담(6월),일자리센터 연계 일자리 상담,"['일자리', '센터', '계', '일자리', '상담']","['일자리', '센터', '연계', '일자리', '상담']"


### 입력 문장에 대한 전처리가 필요하므로 사용했던 함수 작성

In [8]:
# 불용어 처리 
import re

def clean_sentence(sentence) :
    # 날짜, 기수, 차수 제거 
    sentence = re.sub(r"[0-9]+년", r" ", sentence)
    sentence = re.sub(r"[0-9]+차", r" ", sentence)
    sentence = re.sub(r"[0-9]+기", r" ", sentence)
    sentence = re.sub(r"[0-9]+월", r" ", sentence)
    sentence = re.sub(r"[0-9]+일", r" ", sentence)
    sentence = re.sub(r"[0-9]{1,2}.[0-9]{1,2}", r" ", sentence)
    
    # (주) , (요일)
    sentence = re.sub(r"\(+[가-힣]+\)", r" ", sentence)
    #sentence = re.sub(r"[/s]\(.\)[/s]", r" ", sentence)
    
    # 주차, 요일 형식 제거 
    # sentence = re.sub(r"[가-힣]{2}주", r" ", sentence) 
    # "알려주는" 단어에 영향이 생김
    
    sentence = re.sub(r"[가-힣]째주", r" ", sentence) 
#     sentence = re.sub(r"둘째주", r" ", sentence) 
#     sentence = re.sub(r"셋째주", r" ", sentence) 
#     sentence = re.sub(r"넷째주", r" ", sentence) 
    
    sentence = re.sub(r"[가-힣]{1}요일", r" ", sentence)
    
    # 마감 키워드 필요 없음
    sentence = re.sub(r"마감", r" ", sentence)
    
    # 50이라는 숫자 필요 없음 
    sentence = re.sub(r"50", r" ", sentence)
    # 자격증 n급 필요 없을듯 
    sentence = re.sub(r"[0-9]+급", r" ", sentence)
    # n단계도 필요 없을듯 
    sentence = re.sub(r"[0-9]+단계", r" ", sentence)
    
    sentence = re.sub(r"[^0-9가-힣a-zA-Z]", r" ", sentence)
    return sentence

In [9]:
from konlpy.tag import Okt, Komoran, Mecab, Hannanum, Kkma

# 다양한 토크나이저를 사용할 수 있는 함수
def get_tokenizer(tokenizer_name):
    if tokenizer_name == "komoran":
        tokenizer = Komoran()
    elif tokenizer_name == "okt":
        tokenizer = Okt()
    elif tokenizer_name == "mecab":
        tokenizer = Mecab()
    elif tokenizer_name == "hannanum":
        tokenizer = Hannanum()
    else:
        # "kkma":
        tokenizer = Kkma()
        
    return tokenizer

In [93]:
def tokenize(tokenizer_name, original_sent, nouns=False):
    # 미리 정의된 몇 가지 tokenizer 중 하나를 선택
    tokenizer = get_tokenizer(tokenizer_name)

    # tokenizer를 이용하여 original_sent를 토큰화하여 tokenized_sent에 저장하고, 이를 반환합니다.
    sentence = original_sent.replace('\n', '').strip()
    if nouns:       
        # tokenizer.nouns(sentence) -> 명사만 추출
        tokens = tokenizer.nouns(sentence)
    else:
        tokens = tokenizer.morphs(sentence)
    
    return list( map(str, tokens) )

### TF-IDF 활용 문장간 유사도 측정

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
tfidf_mecab = TfidfVectorizer().fit( data["mecab"] )
tfidf_okt = TfidfVectorizer().fit( data["okt"] )

In [13]:
len( tfidf_mecab.vocabulary_ )

4214

In [14]:
mecab_tf = tfidf_mecab.transform( data["mecab"] ).toarray()

In [15]:
mecab_tf.shape

(10692, 4214)

In [16]:
def l1_normalize(v):
  norm = np.sum(v)
  return v / norm

In [53]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_mecab = tfidf_vectorizer.fit_transform( data["mecab"] )
tfidf_norm_l1 = l1_normalize(tfidf_mecab)

In [18]:
tfidf_norm_l1

<10692x4214 sparse matrix of type '<class 'numpy.float64'>'
	with 50928 stored elements in Compressed Sparse Row format>

### cosin 유사도 측정

In [41]:
from sklearn.metrics.pairwise import cosine_similarity

In [42]:
def cosine_similarity_value(vec_1, vec_2):
  return round(cosine_similarity(vec_1, vec_2)[0][0], 3)

In [155]:
## 검색 데이터 입력 
input_data = input()

코딩 교육 


In [156]:
# 임시 데이터 프레임 생성 후 전처리 진행

temp = pd.DataFrame({
    "번호" : "0000",
    "교육명": [input_data],
    "clean_sentence" : clean_sentence(input_data),
     "okt" : ["123"],
     "mecab" : ["123"]
})

temp["okt"] = temp["clean_sentence"].apply(lambda x : tokenize("okt", x, True) )
temp["mecab"] = temp["clean_sentence"].apply(lambda x : tokenize("mecab", x, True) )

temp["okt"] = temp["okt"].apply(lambda x : str(x) for x in temp["okt"])
temp["mecab"] = temp["mecab"].apply(lambda x : str(x) for x in temp["mecab"])

In [157]:
temp

,번호,교육명,clean_sentence,okt,mecab
0,0000,코딩 교육,코딩 교육,"['코딩', '교육']","['코딩', '교육']"


In [158]:
data = pd.concat([data,temp])
data = data.reset_index( drop=True )

In [159]:
data.tail()

,번호,교육명,clean_sentence,okt,mecab,cosin
10689,257,건설업 기초안전보건교육,건설업 기초안전보건교육,"['건설업', '기초', '안전', '보건', '교육']","['건설업', '기초', '안전', '보건', '교육']",0.0
10690,258,취업설계교육 1기,취업설계교육,"['취업', '설계', '교육']","['취업', '설계', '교육']",0.0
10691,259,2023년 직업교육_일반경비원 신임교육 5기,직업교육 일반경비원 신임교육,"['직업', '교육', '일반', '경비원', '신임', '교육']","['직업', '교육', '일반', '경비원', '신임', '교육']",0.0
10692,0000,청소 일자리 ?,청소 일자리,"['청소', '일자리']","['청소', '일자리']",1.0
10693,0000,코딩 교육,코딩 교육,"['코딩', '교육']","['코딩', '교육']",NaN


In [143]:
# data = data.drop([10693], axis=0)

# data.tail()

,번호,교육명,clean_sentence,okt,mecab,cosin
10687,255,취업준비교육 4기,취업준비교육,"['취업', '준비', '교육']","['취업', '준비', '교육']",0.000
10688,256,병원동행매니저 교육 1기,병원동행매니저 교육,"['병원', '동행', '매니저', '교육']","['병원', '동행', '매니저', '교육']",0.000
10689,257,건설업 기초안전보건교육,건설업 기초안전보건교육,"['건설업', '기초', '안전', '보건', '교육']","['건설업', '기초', '안전', '보건', '교육']",0.000
10690,258,취업설계교육 1기,취업설계교육,"['취업', '설계', '교육']","['취업', '설계', '교육']",0.000
10691,259,2023년 직업교육_일반경비원 신임교육 5기,직업교육 일반경비원 신임교육,"['직업', '교육', '일반', '경비원', '신임', '교육']","['직업', '교육', '일반', '경비원', '신임', '교육']",0.395


In [160]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_mecab = tfidf_vectorizer.fit( data["mecab"] )
tfidf_mecab = tfidf_vectorizer.transform( data["mecab"] )
tfidf_norm_l1 = l1_normalize(tfidf_mecab)

In [161]:
target = tfidf_norm_l1[-1]

In [162]:
result = []

for i in tfidf_norm_l1 :
    result.append( cosine_similarity_value(target, i) )

In [163]:
data["cosin"] = result

In [164]:
data.sort_values(["cosin"], ascending=False)[1:6]

,번호,교육명,clean_sentence,okt,mecab,cosin
8672,4766660,[코딩교육 프로그램 강사 - 필수 교육],코딩교육 프로그램 강사 필수 교육,"['코딩', '교육', '프로그램', '강사', '필수', '교육']","['코딩', '교육', '프로그램', '강사', '필수', '교육']",0.665
7940,6167598,[50+자유학교] 앱코딩 (앱인벤터),자유학교 앱코딩,"['자유', '학교', '앱', '코딩']","['자유', '학교', '앱', '코딩']",0.586
3717,25341751,드론코딩및스마트스포츠체험(코딩 교실),드론코딩및스마트스포츠체험 코딩 교실,"['드론', '코딩', '및', '스마트', '스포츠', '체험', '코딩', '교실']","['드론', '코딩', '스마트', '스포츠', '체험', '코딩', '교실']",0.583
3579,26255335,그래서 코딩이 필요합니다(초급),그래서 코딩이 필요합니다,['코딩'],"['코딩', '필요']",0.567
3794,24676804,그래서 코딩이 필요합니다(중급),그래서 코딩이 필요합니다,['코딩'],"['코딩', '필요']",0.567


In [165]:
temp = data.sort_values(["cosin"], ascending=False)[1:6]

temp

,번호,교육명,clean_sentence,okt,mecab,cosin
8672,4766660,[코딩교육 프로그램 강사 - 필수 교육],코딩교육 프로그램 강사 필수 교육,"['코딩', '교육', '프로그램', '강사', '필수', '교육']","['코딩', '교육', '프로그램', '강사', '필수', '교육']",0.665
7940,6167598,[50+자유학교] 앱코딩 (앱인벤터),자유학교 앱코딩,"['자유', '학교', '앱', '코딩']","['자유', '학교', '앱', '코딩']",0.586
3717,25341751,드론코딩및스마트스포츠체험(코딩 교실),드론코딩및스마트스포츠체험 코딩 교실,"['드론', '코딩', '및', '스마트', '스포츠', '체험', '코딩', '교실']","['드론', '코딩', '스마트', '스포츠', '체험', '코딩', '교실']",0.583
3579,26255335,그래서 코딩이 필요합니다(초급),그래서 코딩이 필요합니다,['코딩'],"['코딩', '필요']",0.567
3794,24676804,그래서 코딩이 필요합니다(중급),그래서 코딩이 필요합니다,['코딩'],"['코딩', '필요']",0.567


In [166]:
temp.index

Index([8672, 7940, 3717, 3579, 3794], dtype='int64')

In [203]:
for i,j in zip(data.iloc[temp.index]["교육명"], data.iloc[temp.index]["cosin"]):
    print( i,j )

[코딩교육 프로그램 강사 - 필수 교육] 0.665
[50+자유학교] 앱코딩 (앱인벤터) 0.586
드론코딩및스마트스포츠체험(코딩 교실) 0.583
그래서 코딩이 필요합니다(초급) 0.567
그래서 코딩이 필요합니다(중급) 0.567


In [172]:
print(f"검색 단어 : {input_data}")
print()

for i in data.iloc[temp.index]["교육명"] :
    print( i )

검색 단어 : 코딩 교육 

[코딩교육 프로그램 강사 - 필수 교육]
[50+자유학교] 앱코딩 (앱인벤터)
드론코딩및스마트스포츠체험(코딩 교실)
그래서 코딩이 필요합니다(초급)
그래서 코딩이 필요합니다(중급)


### 검색 단어 관련 교육 추천 기능 함수화

In [173]:
data[::].shape

(10694, 6)

In [207]:
def edu_recommend(input_data, data=data) :
    # 입력 단어에 대한 임시 데이터 프레임 생성    
    temp = pd.DataFrame({
        "번호" : "0000",
        "교육명": [input_data],
        "clean_sentence" : clean_sentence(input_data),
         "okt" : ["123"],
         "mecab" : ["123"]
    })

    temp["okt"] = temp["clean_sentence"].apply(lambda x : tokenize("okt", x, True) )
    temp["mecab"] = temp["clean_sentence"].apply(lambda x : tokenize("mecab", x, True) )

    temp["okt"] = temp["okt"].apply(lambda x : str(x) for x in temp["okt"])
    temp["mecab"] = temp["mecab"].apply(lambda x : str(x) for x in temp["mecab"])
    
    
    # 검색 단어를 포함한 전체 데이터 프레임 
    temp_total_data = data[::]
    
    temp_total_data = pd.concat([temp_total_data,temp])
    temp_total_data = temp_total_data.reset_index( drop=True )
    
    
    # TF-IDF 벡터화 
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_mecab = tfidf_vectorizer.fit( temp_total_data["mecab"] )
    tfidf_mecab = tfidf_vectorizer.transform( temp_total_data["mecab"] )
    tfidf_norm_l1 = l1_normalize(tfidf_mecab)
    
    # 검색 단어 
    target = tfidf_norm_l1[-1]
    
    # 코사인 유사도 적용
    result = []

    for i in tfidf_norm_l1 :
        result.append( cosine_similarity_value(target, i) )
        
    temp_total_data["cosin"] = result
    
    temp = temp_total_data.sort_values(["cosin"], ascending=False)[1:6]
    
    for i,j in zip(data.iloc[temp.index]["교육명"], data.iloc[temp.index]["cosin"]):
        if j > 0 :
            print( i,j )

In [208]:
%%time
input_data = input()

print()
edu_recommend(input_data)
print()

경비

일반경비 신임교육 1기 0.134
<일활동사업>중장년 일반경비신임교육 0.114

CPU times: user 3.63 s, sys: 18.8 ms, total: 3.64 s
Wall time: 5.18 s


In [209]:
%%time
input_data = input()

print()
edu_recommend(input_data)
print()

경호


CPU times: user 3.56 s, sys: 0 ns, total: 3.56 s
Wall time: 4.67 s
